In [2]:
%load_ext autoreload
%autoreload 2

import re
import os
import sys
import nltk
import spacy
import joblib
import skweak
import numpy as np
import pandas as pd

from spacy.tokens import Span
from spacy.tokens import DocBin
from collections import Counter
from spacy.training import Corpus
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from utils.skweak_ner_eval_utils import evaluate

# from skweak import heuristics, gazetteers, aggregation, utils

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
!python --version
#!pip install textblob
!pip freeze > requirements_test.txt
# TODO: add textblob==0.17.1 to 


nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
! python -m textblob.download_corpora

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/andst/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/andst/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/andst/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.3 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# Weakly Supervised Named Entity Tagging with Learnable Logical Rules
# https://universaldependencies.org/format.html
# https://aclanthology.org/2021.acl-long.352.pdf

# Get data from https://github.com/explosion/projects/tree/v3/benchmarks/ud_benchmark
# by using assets command, or downloading https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3105/ud-treebanks-v2.5.tgz

# for each file run with vars.ud_treebank being the treebank you want to use, e.g. ("UD_English-EWT")
# python scripts/copy_files.py train conllu assets/ud-treebanks-v2.5/${vars.ud_treebank}/ corpus/${vars.ud_treebank}/train/
# python -m spacy convert corpus/${vars.ud_treebank}/train/ corpus/${vars.ud_treebank}/ --converter conllu -n 1 -T -C



In [13]:
eng_nlp_sm = spacy.load("en_core_web_sm")
eng_nlp_md = spacy.load("en_core_web_md")

In [57]:
from textblob import TextBlob
from textblob.taggers import PatternTagger

from part_3_eng_pos_tags.scripts.utils import penntreebank2universal


def ground_truth_tagger(doc):
    for token in doc:
        yield token.i, token.i + 1, token.pos_


def nltk_tagger(doc):
    for token in doc:
        if not token.is_punct:
            # Tag token with nltk
            nltk_pos = nltk.pos_tag([token.text], tagset='universal')[0][1]
            yield token.i, token.i + 1, nltk_pos

def nltk_tagger_2(doc):
    for token in doc:
        if not token.is_punct:
            # Tag token with nltk
            nltk_pos = nltk.pos_tag([token.text])[0][1]
            # Map nltk pos tags to ours
            if nltk_pos == "DT":
                yield token.i, token.i+1, "DET"
            elif nltk_pos == "CD":
                yield token.i, token.i+1, "NUM"
            elif nltk_pos == "NNP" or nltk_pos == "NNPS":
                yield token.i, token.i+1,"PROPN"
            elif nltk_pos == "JJ" or nltk_pos == "JJR" or nltk_pos == "JJS":
                yield token.i, token.i+1, "ADJ"
            elif nltk_pos == "NN" or nltk_pos == "NNS":
                yield token.i, token.i+1, "NOUN"
            elif nltk_pos == "VB" or nltk_pos == "VBD" or nltk_pos == "VBG" or nltk_pos == "VBN" or nltk_pos == "VBP" or nltk_pos == "VBZ":
                yield token.i, token.i+1, "VERB"


def textblob_tagger(doc):
    for token in doc:
        if not token.is_punct:
            textblob_pos = TextBlob(token.text, pos_tagger=PatternTagger()).tags
            if len(textblob_pos) > 0:
                yield token.i, token.i + 1, penntreebank2universal(textblob_pos[0][1])


def eng_spacy_tagger_sm(doc):
    other_doc = eng_nlp_sm(doc.text)
    i = 0
    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]


def eng_spacy_tagger_md(doc):
    other_doc = eng_nlp_md(doc.text)

    for token in doc:
        labelled = False
        for other_token in other_doc:
            if other_doc[other_token.i:].text not in doc[token.i:].text:
                continue
            if token.text in other_token.text and not labelled:
                labelled = True
                yield token.i, token.i + 1, other_token.pos_.split("-")[-1]


ground_truth_lf = skweak.heuristics.FunctionAnnotator("v", ground_truth_tagger)

nltk_lf = skweak.heuristics.FunctionAnnotator("nltk", nltk_tagger)
nltk_lf_2 = skweak.heuristics.FunctionAnnotator("nltk_tags", nltk_tagger)

textblob_lf = skweak.heuristics.FunctionAnnotator("textblob", textblob_tagger)
eng_spacy_sm_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_sm", eng_spacy_tagger_sm)
eng_spacy_md_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_md", eng_spacy_tagger_md)


In [ ]:
from part_3_eng_pos_tags.scripts.utils import load_data_split, tag_all


lfs = [nltk_lf, eng_spacy_sm_lf, eng_spacy_md_lf, textblob_lf]

# load training and test data
all_labels = ["DET", "NUM", "ADJ", "NOUN"]
train_docs = load_data_split("train", all_labels, 200)
test_docs = load_data_split("dev", all_labels, 150)

# tag the training documents
train_docs = tag_all(train_docs, lfs)

# train the HMM
mv = skweak.aggregation.MajorityVoter("mv", all_labels)
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm.fit(train_docs)

# tag the test documents
# it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account
test_docs = tag_all(test_docs, lfs + [mv, hmm])


Starting iteration 1
Finished E-step with 200 documents


         1       -8401.0903             +nan


Starting iteration 2
Finished E-step with 200 documents
Starting iteration 3


         2       -8259.8945        +141.1958


Finished E-step with 200 documents
Starting iteration 4


         3       -8248.2718         +11.6227


Finished E-step with 200 documents
Starting iteration 5


         4       -8245.6861          +2.5858


Finished E-step with 200 documents


         5       -8245.1158          +0.5703


In [ ]:
print(test_docs[0].ents, test_docs[0].text)
for tok in test_docs[0]:
    print(tok.text, tok.pos_, tok.ent_type_)

In [52]:
evaluate(test_docs, all_labels, ["nltk", "eng_spacy_sm", "eng_spacy_md", "textblob", "mv", "hmm"])

tok_precision  tok_recall  tok_f1 tok_cee  \
label    proportion model                                                     
DET      22.4 %     eng_spacy_md          0.992       0.956   0.974           
                    eng_spacy_sm          0.991       0.956   0.974           
                    hmm                   0.915       0.992   0.952           
                    mv                    0.885       0.995   0.936           
                    nltk                  0.913       0.982   0.946           
                    textblob              0.884       0.982   0.930           
NOUN     48.2 %     eng_spacy_md          0.931       0.895   0.912           
                    eng_spacy_sm          0.922       0.889   0.906           
                    hmm                   0.905       0.908   0.906           
                    mv                    0.612       0.934   0.740           
                    nltk                  0.491       0.969   0.652           
                    textblob              0.570       0.940   0.710           
NUM      4.9 %      eng_spacy_md          0.961       0.884   0.920           
                    eng_spacy_sm          0.952       0.887   0.918           
                    hmm                   0.952       0.949   0.950           
                    mv                    0.946       0.988   0.966           
                    nltk                  0.930       0.946   0.938           
                    textblob              0.964       0.952   0.958           
PROPN    24.4 %     eng_spacy_md          0.810       0.920   0.862           
                    eng_spacy_sm          0.811       0.914   0.860           
                    hmm                   0.822       0.900   0.860           
                    mv                    0.836       0.892   0.864           
                    nltk                  0.000       0.000   0.000           
                    textblob              0.000       0.000   0.000           
macro               eng_spacy_md          0.924       0.914   0.918           
                    eng_spacy_sm          0.919       0.912   0.916           
                    hmm                   0.899       0.937   0.918           
                    mv                    0.820       0.952   0.882           
                    nltk                  0.584       0.724   0.646           
                    textblob              0.604       0.718   0.656           
micro               eng_spacy_md          0.912       0.914   0.912  11.668   
                    eng_spacy_sm          0.908       0.910   0.908  11.668   
                    hmm                   0.888       0.927   0.908   0.347   
                    mv                    0.723       0.940   0.818   0.289   
                    nltk                  0.590       0.734   0.654  11.668   
                    textblob              0.659       0.720   0.688  11.668   
weighted            eng_spacy_md          0.917       0.914   0.916           
                    eng_spacy_sm          0.912       0.910   0.910           
                    hmm                   0.889       0.927   0.908           
                    mv                    0.744       0.940   0.830           
                    nltk                  0.487       0.734   0.586           
                    textblob              0.520       0.720   0.604           

                                 tok_acc coverage  ent_precision  ent_recall  \
label    proportion model                                                      
DET      22.4 %     eng_spacy_md                           0.992       0.956   
                    eng_spacy_sm                           0.991       0.956   
                    hmm                                    0.915       0.992   
                    mv                                     0.885       0.995   
                    nltk                                   0.913       0.982 

In [53]:
print("test")

test


In [309]:
from skweak.utils import docbin_writer

for doc in train_docs:
    doc.set_ents(doc.spans.get("nltk", []))

test_docs = load_data_split("dev", all_labels, 1500)

# Save the training and development data.
os.makedirs("tmp", exist_ok=True)
docbin_writer(train_docs, "./tmp/train.spacy")
docbin_writer(test_docs, "./tmp/dev.spacy")

Write to ./tmp/train.spacy...done
Write to ./tmp/dev.spacy...done


In [219]:
for tok in test_docs[0]:
    print(tok, tok.pos_, tok.ent_type_)
print("")
for tok in dev_eval_docs[0]:
    print(tok, tok.ent_type_)

From ADP 
the DET DET
AP PROPN PROPN
comes VERB 
this DET DET
story NOUN NOUN
: PUNCT 

From 
the 
AP 
comes 
this 
story 
: 


In [23]:
test_docs = load_data_split("dev", all_labels, 1500)


In [24]:
test_docs[0].ents

(the, AP, this, story)

In [42]:
d = eng_spacy_sm_lf(test_docs[2])
d.set_ents(d.spans.get("eng_spacy_sm", []))

0 Bush Bush 0 0
0 nominated nominated 1 1
0 Jennifer Jennifer 2 2
0 M. M. 3 3
0 Anderson Anderson 4 4
0 for for 5 5
0 a a 6 6
0 15 15 7 7
0 - - 8 8
0 year year 9 9
0 term term 10 10
0 as as 11 11
0 associate associate 12 12
0 judge judge 13 13
0 of of 14 14
0 the the 15 15
0 Superior Superior 16 16
0 Court Court 17 17
0 of of 18 18
0 the the 19 19
0 District District 20 20
0 of of 21 21
0 Columbia Columbia 22 22
0 , , 23 23
0 replacing replacing 24 24
0 Steffen Steffen 25 25
0 W. W. 26 26
0 Graae Graae 27 27
0 . . 28 28


In [43]:
set(d.spans.get("eng_spacy_sm", []))

{Bush,
 nominated,
 Jennifer,
 M.,
 Anderson,
 for,
 a,
 15,
 -,
 year,
 term,
 as,
 associate,
 judge,
 of,
 the,
 Superior,
 Court,
 of,
 the,
 District,
 of,
 Columbia,
 ,,
 replacing,
 Steffen,
 W.,
 Graae,
 .}

In [38]:
for a in d.spans.get("eng_spacy_sm", []):
    print(a.text, a.start, a.end, a.label_)

Bush 0 1 PROPN
nominated 1 2 VERB
Jennifer 2 3 PROPN
M. 3 4 PROPN
Anderson 4 5 PROPN
for 5 6 ADP
a 6 7 DET
a 6 7 NOUN
a 6 7 ADP
a 6 7 ADJ
a 6 7 PROPN
a 6 7 VERB
a 6 7 PROPN
15 7 8 NUM
- 8 9 PUNCT
year 9 10 NOUN
term 10 11 NOUN
as 11 12 ADP
as 11 12 ADJ
associate 12 13 ADJ
judge 13 14 NOUN
of 14 15 ADP
of 14 15 ADP
of 14 15 ADP
the 15 16 DET
the 15 16 DET
Superior 16 17 PROPN
Court 17 18 PROPN
of 18 19 ADP
of 18 19 ADP
the 19 20 DET
District 20 21 PROPN
of 21 22 ADP
Columbia 22 23 PROPN
, 23 24 PUNCT
replacing 24 25 VERB
Steffen 25 26 PROPN
W. 26 27 PROPN
Graae 27 28 PROPN
. 28 29 PUNCT


In [306]:
d.spans.get("nltk", [])

[Bush, nominated, Jennifer, M., Anderson, for, a, 15, year, term, as, associate, judge, of, the, Superior, Court, of, the, District, of, Columbia, replacing, Steffen, W., Graae]

In [285]:
d.text

'Bush nominated Jennifer M. Anderson for a 15-year term as associate judge of the Superior Court of the District of Columbia, replacing Steffen W. Graae. '

In [313]:
from spacy.training import Example
from spacy.scorer import Scorer

dev_eval_docs = load_data_split("dev", all_labels, 1500)

i = 0
for doc in dev_eval_docs:
    doc = nltk_lf(doc)
    doc.set_ents(doc.spans.get("nltk", []))

scorer_object = Scorer()
scores = scorer_object.score(
    [Example(dev_eval_docs[i], test_docs[i]) for i in range(0, len(test_docs))]
)

pd.DataFrame(
    [{k: v for k, v in scores.items() if k in ["ents_p", "ents_r", "ents_f", "score"]}]
).round(3)

,ents_p,ents_r,ents_f
0,0.283,0.734,0.408


In [311]:
!spacy init config - --lang xx --pipeline ner --optimize accuracy | \
    spacy train - \
    --training.max_steps 400 \
    --system.seed 42 \
    --paths.train./tmp/train.spacy \
    --paths.dev./tmp/dev.spacy \
    --output./tmp/model

ℹ Saving to output directory: tmp/model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     82.22    9.03    6.64   14.11    0.09
  0     200         47.03   5068.15   39.32   27.23   70.70    0.39
  1     400         64.09   1508.40   39.53   27.39   71.03    0.40
✔ Saved pipeline to output directory
tmp/model/model-last
